In [7]:
#First of all we define our map
#we put D for dot/ E for empty/ G for ghost/ W for wall/ A for agent

plan = [['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
       ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
       ['W', 'W', 'E', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'W', 'W'],
       ['W', 'W', 'D', 'W', 'W', 'W', 'D', 'W', 'W', 'W', 'D', 'W', 'W'],
       ['W', 'W', 'D', 'W', 'D', 'D', 'D', 'D', 'D', 'W', 'D', 'W', 'W'],
       ['W', 'W', 'D', 'D', 'D', 'W', 'E', 'W', 'D', 'D', 'D', 'W', 'W'],
       ['W', 'W', 'D', 'W', 'D', 'W', 'E', 'W', 'D', 'W', 'D', 'W', 'W'],
       ['W', 'W', 'D', 'W', 'D', 'D', 'W', 'D', 'D', 'W', 'D', 'W', 'W'],
       ['W', 'W', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'W', 'W'],
       ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
       ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W']]

In [8]:
#Here we build all states in which we have at most one ghost and agent is always in the middle
#and we used backtracking to build it.
#And we checked if it makes all possible states by counting all possible states manually and check it
#with our states length.

all_states = []
def Build_all_states(i, cur_state, have_G):
    if(i == 13):
        all_states.append(cur_state)
        return
    
    if(i == 6):
        cur_state += 'A'
        Build_all_states(i + 1, cur_state, have_G)
        return
    
    possible_chars = ['D', 'E', 'W']
    if(have_G == False):
        possible_chars.append('G')
    for char in possible_chars:
        new_state = cur_state + char
        Build_all_states(i + 1, new_state, (have_G or (char == 'G')))
    return

Build_all_states(0, "", False)
all_states.append("L")
print(len(all_states))
print((12 * (3 ** 11)) + 3 ** 12 + 1)

2657206
2657206


In [9]:
#Now it's time to define a function which finds next state
#But first we need a function to develop random walk for our ghost
#and then with finding our agent's and ghost's next location
#we find if out agent loses or not and if it doesn't lose
#we return it's new location with it's reward

def find_next_G_loc(G_loc, cur_plan):
    moves = ['L', 'R', 'U', 'D']
    act = moves[np.random.randint(4)]
    
    new_G_loc = G_loc.copy()
    if(act == 'R' and cur_plan[G_loc[0]][G_loc[1] + 1] != 'W'):
        new_G_loc[1] += 1
    elif(act == 'L' and cur_plan[G_loc[0]][G_loc[1] - 1] != 'W'):
        new_G_loc[1] -= 1
    elif(act == 'U' and cur_plan[G_loc[0] - 1][G_loc[1]] != 'W'):
        new_G_loc[0] -= 1
    elif(act == 'D' and cur_plan[G_loc[0] + 1][G_loc[1]] != 'W'):
        new_G_loc[0] += 1
    
    return new_G_loc
    

def find_next_state(cur_plan, A_loc, G_loc, act):
    new_A_loc = A_loc.copy()
    if(act == 'R' and cur_plan[A_loc[0]][A_loc[1] + 1] != 'W'):
        new_A_loc[1] += 1
    elif(act == 'L' and cur_plan[A_loc[0]][A_loc[1] - 1] != 'W'):
        new_A_loc[1] -= 1
    elif(act == 'U' and cur_plan[A_loc[0] - 1][A_loc[1]] != 'W'):
        new_A_loc[0] -= 1
    elif(act == 'D' and cur_plan[A_loc[0] + 1][A_loc[1]] != 'W'):
        new_A_loc[0] += 1
    new_G_loc = find_next_G_loc(G_loc.copy(), cur_plan)
    
    if ((new_A_loc[0] == new_G_loc[0] and new_A_loc[1] == new_G_loc[1])
        or ((A_loc[0] == new_G_loc[0] and A_loc[1] == new_G_loc[1]) 
            and (new_A_loc[0] == G_loc[0] and new_A_loc[1] == G_loc[1]))):
        reward = -10
        new_state = "L"
        return new_state, reward, new_A_loc, new_G_loc, cur_plan
    
    reward = 0
    if(cur_plan[new_A_loc[0]][new_A_loc[1]] == 'D'):
        cur_plan[new_A_loc[0]][new_A_loc[1]] = 'E'
        reward = 1
        
    new_state = ""
    if(new_A_loc[0] - 2 == new_G_loc[0] and new_A_loc[1] == new_G_loc[1]):
        new_state += 'G'
    else:
        new_state += cur_plan[new_A_loc[0] - 2][new_A_loc[1]]
    for curi in range(new_A_loc[0] - 1, new_A_loc[0] + 2):
        L = new_A_loc[1] - 1 - (curi == new_A_loc[0])
        R = new_A_loc[1] + 2 + (curi == new_A_loc[0])
        for curj in range(L, R):
            if(curi == new_A_loc[0] and curj == new_A_loc[1]):
                new_state += 'A'
            elif(curi == new_G_loc[0] and curj == new_G_loc[1]):
                new_state += 'G'
            else:
                new_state += cur_plan[curi][curj]
    if(new_A_loc[0] + 2 == new_G_loc[0] and new_A_loc[1] == new_G_loc[1]):
        new_state += 'G'
    else:
        new_state += cur_plan[new_A_loc[0] + 1][new_A_loc[1]]

    return new_state, reward, new_A_loc, new_G_loc, cur_plan

In [10]:
#Here we hash our states into number from 0 to number of states
#to achieve them faster and learn faster
all_states_hash = ["#" for i in range(len(all_states))]
def get_state_id(state):
    cur = 0; B = 256; M = len(all_states)
    for i in range(len(state)):
        cur = ((cur * B) + ord(state[i])) % M
        
    while(all_states_hash[cur] != '#' and all_states_hash[cur] != state):
        cur = (cur + 1) % M
        
    if(all_states_hash[cur] == '#'):
        all_states_hash[cur] = state
        
    return cur

In [40]:
#Here we define our Q function 
#We learn our Q function episodic and we do 30000 episodes to make sure our q function is learnt properly
#we put a dynamic probability for it's movements
#And after fixing each of it's movements we calculate it's next state and reward and base on these two
#we update our Q table and also each 1000 episode, we print it's average achieved reward to check if it's converging

import numpy as np
def Qlearn(all_states, epsilon, alpha, gamma, episode_num, flag):
    Q = np.zeros((len(all_states), 4))
    action_space = ['L', 'R', 'U', 'D']

    average_reward = 0
    number_of_lose = 0
    for episode in range(1, episode_num + 1):
        if(epsilon > 0):
            epsilon -= 1 / 20000

        state = "WWWWWWADDWDWD"
        state_id = get_state_id(state)
        cur_plan = [[plan[i][j] for j in range(len(plan[0]))] for i in range(len(plan))]
        loc_A = [2, 2]
        loc_G = [6, 6]

        for i in range(100):
            if np.random.rand() < epsilon:
                action = np.random.randint(4)
            else:
                action = np.argmax(Q[state_id, :])

            next_state, reward, loc_A, loc_G, cur_plan = find_next_state(cur_plan, loc_A.copy(), loc_G.copy(), action_space[action])
            next_state_id = get_state_id(next_state)

            Q[state_id, action] = (1 - alpha) * Q[state_id, action] + alpha * (reward + gamma * np.max(Q[next_state_id, :]))

            average_reward += reward
            state = next_state
            state_id = get_state_id(state)
            if(state == 'L'):
                number_of_lose += 1
                break

        if(episode % 1000 == 0):
            average_reward /= 1000
            if(flag):
                print("episode", episode)
                print("average_reward", average_reward)
                print("number_of_lose", number_of_lose)
            if(episode != episode_num):
                average_reward = 0
                number_of_lose = 0
    return Q, average_reward, number_of_lose

In [41]:
Q_table, average_reward, number_of_lose = Qlearn(all_states, 1, 0.5, 0.6, 30000, True)

episode 1000
average_reward 7.332
number_of_lose 482
episode 2000
average_reward 8.898
number_of_lose 517
episode 3000
average_reward 11.323
number_of_lose 503
episode 4000
average_reward 13.63
number_of_lose 509
episode 5000
average_reward 15.84
number_of_lose 512
episode 6000
average_reward 18.116
number_of_lose 508
episode 7000
average_reward 19.484
number_of_lose 512
episode 8000
average_reward 21.518
number_of_lose 500
episode 9000
average_reward 23.739
number_of_lose 484
episode 10000
average_reward 24.788
number_of_lose 456
episode 11000
average_reward 26.192
number_of_lose 444
episode 12000
average_reward 27.757
number_of_lose 420
episode 13000
average_reward 29.153
number_of_lose 397
episode 14000
average_reward 30.47
number_of_lose 359
episode 15000
average_reward 32.405
number_of_lose 314
episode 16000
average_reward 33.636
number_of_lose 268
episode 17000
average_reward 34.838
number_of_lose 243
episode 18000
average_reward 35.62
number_of_lose 194
episode 19000
average_rew

In [42]:
#Here we check which combination of alpha and gamma is the best for our model
alpha_list = [0.1, 0.5, 0.8]
gamma_list = [0.25, 0.5, 0.9, 1]

max_average_reward = 0
max_alpha = 0
max_gamma = 0

for alpha in alpha_list:
    for gamma in gamma_list:
        Q_table, average_reward, number_of_lose = Qlearn(all_states, 1, alpha, gamma, 30000, False)
        print(alpha, gamma, average_reward, number_of_lose)
        if(average_reward > max_average_reward):
            max_average_reward = average_reward
            max_alpha = alpha
            max_gamma = gamma

print(max_alpha, max_gamma, max_average_reward)

0.1 0.25 42.252 7
0.1 0.5 41.972 8
0.1 0.9 40.882 4
0.1 1 6.307 1
0.5 0.25 40.005 3
0.5 0.5 41.647 2
0.5 0.9 40.979 6
0.5 1 5.902 0
0.8 0.25 38.521 2
0.8 0.5 38.293 3
0.8 0.9 38.717 10
0.8 1 4.379 5
0.1 0.25 42.252


In [45]:
#Here we build or adjancy list by saving transitions of our model learning again
#in the adjancy_list variable which is a list of dictionaries and each dictionary is
#for one of the states and it has for each of it's actions, values of next state and it's reward
adjancy_list = [{} for i in range(len(all_states))]
Q = np.zeros((len(all_states), 4))
action_space = ['L', 'R', 'U', 'D']
alpha = max_alpha
gamma = max_gamma
epsilon = 1
episode_num = 30000

average_reward = 0
number_of_lose = 0
for episode in range(1, episode_num + 1):
    state = "WWWWWWADDWDWD"
    state_id = get_state_id(state)
    cur_plan = [[plan[i][j] for j in range(len(plan[0]))] for i in range(len(plan))]
    loc_A = [2, 2]
    loc_G = [6, 6]

    for i in range(100):
        if np.random.rand() < epsilon:
            action = np.random.randint(4)
        else:
            action = np.argmax(Q[state_id, :])
        next_state, reward, loc_A, loc_G, cur_plan = find_next_state(cur_plan, loc_A.copy(), loc_G.copy(), action_space[action])
        adjancy_list[state_id][action_space[action]] = (next_state, reward)
        next_state_id = get_state_id(next_state)
        Q[state_id, action] = (1 - alpha) * Q[state_id, action] + alpha * (reward + gamma * np.max(Q[next_state_id, :]))
        state = next_state
        state_id = get_state_id(state)
        if(state == 'L'):
            break


In [172]:
#Our start state's adjances and it's transition rewards
print('L', adjancy_list[get_state_id("WWWWWWADDWDWD")]['L'])
print('R', adjancy_list[get_state_id("WWWWWWADDWDWD")]['R'])
print('D', adjancy_list[get_state_id("WWWWWWADDWDWD")]['D'])
print('U', adjancy_list[get_state_id("WWWWWWADDWDWD")]['U'])

L ('WWWWWWADDWDWD', 0)
R ('WWWWWEADDDWWW', 1)
D ('WWEDWWAWWWDWD', 1)
U ('WWWWWWADDWDWD', 0)


In [49]:
#Here we learn our Q function again to save our Q table
Q_table, average_reward, number_of_lose = Qlearn(all_states, 1, max_alpha, max_gamma, 30000, False)

In [53]:
#Here we save our Q table in 'Q_Table.txt' file
np.savetxt('Q_Table.txt', Q_table, fmt='%d', delimiter=' ')

In [103]:
#Here we devise another plan to test our Q function in it
new_plan = [['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
            ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
            ['W', 'W', 'E', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'W', 'W'],
            ['W', 'W', 'D', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'D', 'W', 'W'],
            ['W', 'W', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'W', 'W'],
            ['W', 'W', 'D', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'D', 'W', 'W'],
            ['W', 'W', 'D', 'D', 'D', 'D', 'E', 'D', 'D', 'D', 'D', 'W', 'W'],
            ['W', 'W', 'D', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'D', 'W', 'W'],
            ['W', 'W', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'W', 'W'],
            ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
            ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W']]

In [170]:
#And here we move on our new plan and save how much reward we achieve
#Also we use a probiblity distirbution for deciding our action again
#To don't get stuck in new states.
total_sum = 0
for i in range(10):
    action_space = ['L', 'R', 'U', 'D']
    reward_sum = 0
    state = "WWWWWWADDWDWD"
    state_id = get_state_id(state)
    cur_plan = [[new_plan[i][j] for j in range(len(plan[0]))] for i in range(len(plan))]
    loc_A = [2, 2]
    loc_G = [6, 6]
    epsilon = 0.3

    while(state != "L" and reward_sum < 40):
        if np.random.rand() < epsilon:
            action = np.random.randint(4)
        else:
            action = np.argmax(Q[state_id, :])

        next_state, reward, loc_A, loc_G, cur_plan = find_next_state(cur_plan, loc_A.copy(), loc_G.copy(), action_space[action])
        next_state_id = get_state_id(next_state)

        if(reward > 0):
            reward_sum += reward
        state = next_state
        state_id = get_state_id(state)

    print(reward_sum)
    total_sum += reward_sum
print("Average = ", total_sum / 10)

28
29
6
15
5
28
27
12
13
9
Average =  17.2
